In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

#Importing the shapefile
br = gpd.read_file('../data/roads.shx')

DriverError: Unable to open ../data/roads.shp or ../data/roads.SHP.

In [94]:
br.head()

,osm_id,name,ref,type,oneway,bridge,maxspeed,geometry
0,8656088,Park Rd,NaN,residential,0,0,NaN,"LINESTRING (90.42258 23.79758, 90.42238 23.798..."
1,8656089,Embassy Rd,NaN,residential,0,0,NaN,"LINESTRING (90.41891 23.79768, 90.42034 23.798..."
2,8656151,Dutabash Rd,NaN,residential,0,0,NaN,"LINESTRING (90.41857 23.79918, 90.41941 23.799..."
3,8656152,Road 1,NaN,residential,0,0,NaN,"LINESTRING (90.41926 23.80001, 90.42059 23.800..."
4,8656154,Road 2,NaN,residential,0,0,NaN,"LINESTRING (90.42049 23.80098, 90.42088 23.801..."


In [95]:
#Only selecting primary roads to save computing time
primary_df = br[br['type'] == 'primary']

In [ ]:
#Importing the intersections from the data cleaning file
intersections = pd.read_csv('../data/intersections.csv')

In [ ]:
#Dropping duplicates by removing N1 and N2
intersections = intersections.drop(intersections[intersections['road'] == 'N1'].index)
intersections = intersections.drop(intersections[intersections['road'] == 'N2'].index)
intersections = intersections.reset_index(drop = True)
intersections

In [ ]:
#Finding N2
N2 = br[br['ref'] == 'N 2']
N2.ref = 'N2'
#Finding N1 by manually adding the last 2 parts
N1 = br[br['ref'] == 'N 1']
N1_2 = br[br['ref'] == 'N1']
Chit = br[br['ref'] == "Chittagong Cox's"]
Coxs = br[br['ref'] == "Chittagog Coxs B"]
#Removing sideroads
Coxs = Coxs[Coxs['name'] != 'M A Aziz Road']
Coxs = Coxs[Coxs['name'] != 'O.R. Nizam Road']
Coxs = Coxs[Coxs['name'] != 'Sk.Mujib Road']
#Plotting N1 and N2 together
fig, ax = plt.subplots(1,1)
N2.plot(ax=ax, color = 'blue')
N1.plot(ax=ax, color = 'blue')
N1_2.plot(ax=ax, color = 'blue')
Chit.plot(ax=ax, color = 'blue')
Coxs.plot(ax=ax, color = 'blue')
N1 = pd.concat([N1, N1_2, Chit, Coxs]).reset_index(drop = True)
N1.ref = 'N1'
main = pd.concat([N1, N2]).reset_index(drop = True)


In [ ]:
main

In [ ]:
#Finding all intersections between the two dataframes
intersect = []
for index, row in main.iterrows():
    for i, r in primary_df.iterrows():
        if row['geometry'].intersects(r['geometry']):
            a = [row.ref, row['name'], r.ref, r['name'], r.geometry.intersection(row.geometry)]
            intersect.append(a)
columns = ['Main road ref', 'Main road name', 'Side road ref', 'Side road name', 'geometry']
intersect_df = pd.DataFrame(intersect, columns = columns)
intersect_df

In [ ]:
#Dropping duplicate locations
intersect_df = intersect_df.drop_duplicates(['geometry']).reset_index(drop = True)

In [ ]:
#Checking which roads intersect with N1 and N2
intersect_df['Side road ref'].unique()

In [ ]:
#Removing intersections caused by N1 and N2
mainroads = ['N1', 'N 1', 'N2', 'N 2']
intersect_df = intersect_df.drop(intersect_df[intersect_df['Side road ref'].isin(mainroads)].index).reset_index(drop = True)
intersect_df

In [ ]:
#Transforming the lat and lon to GeoPandas points
intersections['geometry'] = gpd.points_from_xy(intersections.lon, intersections.lat, crs="EPSG:4326")
intersections

In [ ]:
#Finding the distances between the intersections from data cleaning and the intersections from the shapefile
smallest_distance = []
for i in intersections.geometry:
    for j in intersect_df.geometry:
        distances = []
        distances.append(i.distance(j))
    smallest_distance.append(min(distances))
smallest_distance

In [ ]:
#Finding the average distance 
np.mean(smallest_distance)